# Wiktionary Etymology Database Analysis

This notebook provides SQL queries to explore the Wiktionary etymology database.

**Database Schema:**
```sql
CREATE TABLE words (
  word TEXT NOT NULL,
  language TEXT NOT NULL,
  lang_code TEXT,
  pos TEXT,
  etymology_index INTEGER NOT NULL DEFAULT 0,
  etymology_text TEXT,
  etymology_templates TEXT,  -- JSON array
  categories TEXT,           -- JSON array
  senses TEXT,               -- JSON array
  full_data TEXT,            -- Complete JSON
  PRIMARY KEY (word, language, pos, etymology_index)
);
```

In [ ]:
# Setup
import sqlite3
import pandas as pd
import json

# Connect to database
DB_PATH = 'data/wiktionary.db'
conn = sqlite3.connect(DB_PATH)

# Helper function to run queries and display results
def query(sql, params=None):
    """Execute SQL query and return pandas DataFrame"""
    if params:
        return pd.read_sql_query(sql, conn, params=params)
    return pd.read_sql_query(sql, conn)

print(f"Connected to {DB_PATH}")

## 1. Database Statistics

In [ ]:
# Overall database statistics
stats_query = """
SELECT 
    COUNT(*) as total_entries,
    COUNT(DISTINCT word) as unique_words,
    COUNT(DISTINCT language) as unique_languages,
    COUNT(CASE WHEN etymology_text IS NOT NULL THEN 1 END) as entries_with_etymology,
    COUNT(CASE WHEN etymology_index > 0 THEN 1 END) as entries_with_multiple_etymologies
FROM words
"""

query(stats_query)

In [ ]:
# Top 20 languages by entry count
top_languages = """
SELECT 
    language,
    COUNT(*) as entry_count,
    COUNT(CASE WHEN etymology_text IS NOT NULL THEN 1 END) as with_etymology,
    ROUND(COUNT(CASE WHEN etymology_text IS NOT NULL THEN 1 END) * 100.0 / COUNT(*), 1) as etymology_percentage
FROM words
GROUP BY language
ORDER BY entry_count DESC
LIMIT 20
"""

query(top_languages)

## 2. Exploring Specific Words

In [ ]:
# Find all entries for a specific word (example: "cheese")
word_lookup = """
SELECT 
    word,
    language,
    pos,
    etymology_index,
    SUBSTR(etymology_text, 1, 150) as etymology_preview
FROM words
WHERE word = 'cheese'
    AND language = 'English'
ORDER BY pos, etymology_index
"""

query(word_lookup)

In [ ]:
# Get full details for a specific word entry
word = 'cheese'
language = 'English'
pos = 'noun'
etym_index = 0

result = query("""
    SELECT * FROM words 
    WHERE word = ? AND language = ? AND pos = ? AND etymology_index = ?
""", params=(word, language, pos, etym_index))

if len(result) > 0:
    entry = result.iloc[0]
    print(f"Word: {entry['word']}")
    print(f"Language: {entry['language']}")
    print(f"Part of Speech: {entry['pos']}")
    print(f"Etymology Index: {entry['etymology_index']}")
    print(f"\nEtymology:\n{entry['etymology_text']}")
    
    if entry['etymology_templates']:
        templates = json.loads(entry['etymology_templates'])
        print(f"\nEtymology Templates ({len(templates)}):")
        for t in templates[:5]:
            print(f"  - {t.get('name', 'unknown')}: {t.get('args', {})}")
    
    if entry['senses']:
        senses = json.loads(entry['senses'])
        print(f"\nDefinitions ({len(senses)}):")
        for i, sense in enumerate(senses[:3], 1):
            gloss = sense.get('glosses', [''])[0] or sense.get('raw_glosses', [''])[0]
            print(f"  {i}. {gloss}")

## 3. Etymology Relationships

In [ ]:
# Find words with the most etymology templates (most connections)
most_connected = """
SELECT 
    word,
    language,
    pos,
    LENGTH(etymology_templates) - LENGTH(REPLACE(etymology_templates, '{', '')) as template_count,
    SUBSTR(etymology_text, 1, 100) as etymology_preview
FROM words
WHERE etymology_templates IS NOT NULL
    AND language = 'English'
ORDER BY template_count DESC
LIMIT 20
"""

query(most_connected)

In [ ]:
# Find words borrowed from Latin
latin_borrowings = """
SELECT 
    word,
    language,
    pos,
    SUBSTR(etymology_text, 1, 150) as etymology_preview
FROM words
WHERE language = 'English'
    AND etymology_templates LIKE '%"name":"bor"%'
    AND etymology_templates LIKE '%"2":"la"%'
LIMIT 20
"""

query(latin_borrowings)

In [ ]:
# Find Proto-Indo-European roots
pie_roots = """
SELECT 
    word,
    pos,
    SUBSTR(etymology_text, 1, 200) as etymology_preview
FROM words
WHERE language = 'Proto-Indo-European'
    OR lang_code = 'ine-pro'
ORDER BY word
LIMIT 20
"""

query(pie_roots)

## 4. Words with Multiple Etymologies

In [ ]:
# Find English words with multiple etymologies
multiple_etymologies = """
SELECT 
    word,
    pos,
    COUNT(*) as etymology_count
FROM words
WHERE language = 'English'
GROUP BY word, pos
HAVING etymology_count > 1
ORDER BY etymology_count DESC, word
LIMIT 20
"""

query(multiple_etymologies)

In [ ]:
# Compare different etymologies for a word
word_to_compare = 'cheese'

compare_etymologies = f"""
SELECT 
    etymology_index,
    pos,
    SUBSTR(etymology_text, 1, 200) as etymology_preview,
    json_extract(senses, '$[0].glosses[0]') as first_definition
FROM words
WHERE word = '{word_to_compare}'
    AND language = 'English'
ORDER BY pos, etymology_index
"""

query(compare_etymologies)

## 5. Language Families and Proto-Languages

In [ ]:
# Find all proto-languages in database
proto_languages = """
SELECT 
    language,
    lang_code,
    COUNT(*) as entry_count
FROM words
WHERE language LIKE 'Proto-%' OR lang_code LIKE '%-pro'
GROUP BY language, lang_code
ORDER BY entry_count DESC
"""

query(proto_languages)

In [ ]:
# Find Old/Middle/Modern language variants
language_evolution = """
SELECT 
    language,
    COUNT(*) as entry_count
FROM words
WHERE language LIKE '%English%'
    OR language LIKE '%French%'
    OR language LIKE '%German%'
GROUP BY language
ORDER BY language
"""

query(language_evolution)

## 6. Full-Text Search Examples

In [ ]:
# Find words by etymology pattern (example: from Greek)
from_greek = """
SELECT 
    word,
    language,
    pos,
    SUBSTR(etymology_text, 1, 150) as etymology_preview
FROM words
WHERE language = 'English'
    AND etymology_text LIKE '%Ancient Greek%'
    AND word NOT LIKE '-%'
ORDER BY RANDOM()
LIMIT 20
"""

query(from_greek)

In [ ]:
# Search by definition content
search_definition = """
SELECT 
    word,
    language,
    pos,
    json_extract(senses, '$[0].glosses[0]') as definition
FROM words
WHERE language = 'English'
    AND senses LIKE '%cheese%'
    AND word != 'cheese'
LIMIT 20
"""

query(search_definition)

## 7. Custom Analysis

Use this cell to write your own SQL queries:

In [ ]:
# Your custom query here
custom_query = """
SELECT * FROM words
WHERE word = 'run' AND language = 'English'
LIMIT 10
"""

query(custom_query)

In [ ]:
# Close connection when done
conn.close()
print("Connection closed")